<a href="https://colab.research.google.com/github/leman-cap13/my_projects/blob/main/emotions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d sujaykapadnis/emotion-recognition-dataset

In [ ]:
import zipfile

with zipfile.ZipFile('/content/emotion-recognition-dataset.zip','r') as zip_ref:
  zip_ref.extractall()

In [ ]:
import shutil

shutil.rmtree('/content/dataset/Ahegao')

In [ ]:
import tensorflow as tf
img_size=(224,224)

data_path='/content/dataset'


train_data=tf.keras.preprocessing.image_dataset_from_directory(data_path,
                                                                label_mode='categorical',
                                                                image_size=img_size,
                                                                subset='training',
                                                                seed=42,
                                                                validation_split=0.2,
                                                                crop_to_aspect_ratio=True)


test_data=tf.keras.preprocessing.image_dataset_from_directory(data_path,
                                                                label_mode='categorical',
                                                                image_size=img_size,
                                                                subset='validation',
                                                                seed=42,
                                                                validation_split=0.2,
                                                                crop_to_aspect_ratio=True)

In [ ]:
class_names=train_data.class_names

In [ ]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomHeight(0.2),
    tf.keras.layers.RandomWidth(0.2),
    tf.keras.layers.RandomContrast(0.2),
    tf.keras.layers.RandomZoom(0.2)
])

In [ ]:
inputs=tf.keras.layers.Input(shape=(224,224,3))

data_aug=data_augmentation(inputs)
res=tf.keras.layers.Resizing(224,224)(data_aug)

x = tf.keras.applications.xception.preprocess_input(res)
base_model=tf.keras.applications.Xception(weights='imagenet',include_top=False, input_tensor=x)

base_model.trainable=True
for layer in base_model.layers[:100]:
  layer.trainbale=False

avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
x = tf.keras.layers.Dense(256, activation='relu')(avg)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(5, activation='softmax')(x)

model=tf.keras.Model(inputs,output)

In [ ]:
initial_learning_rate=0.0001
lr_schedule=tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=596,
    decay_rate=0.96
)

In [ ]:
early_stopping=tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True

)

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Nadam(learning_rate=lr_schedule),
              metrics=['accuracy', 'f1_score'])

In [ ]:
checkpoint_cb=tf.keras.callbacks.ModelCheckpoint(
    'emotions_detector.keras',
    save_best_only=True,
)

In [ ]:
model.fit(train_data, validation_data=test_data, epochs=10,callbacks=[early_stopping, checkpoint_cb ])

In [ ]:
import tensorflow as tf

In [ ]:
model.evaluate(train_data)

In [ ]:
model.evaluate(test_data)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

In [ ]:
def pred_and_plot(model,image_path):
  img=mpimg.imread(image_path)
  img=tf.image.resize(img,size=[224,224])
  img=img[tf.newaxis,...]
  pred=model.predict(img)

  plt.figure(figsize=(8,5))
  plt.subplot(1,2,1)
  plt.imshow(img[0]/255)
  plt.title(f"Prediction:{class_names[np.argmax(pred)]}")
  plt.axis(False)
  plt.subplot(1,2,2)
  plt.bar(class_names,pred[0])
  plt.title('probability')
  plt.show()

pred_and_plot(model,'/content/vvg.jpg')

In [ ]:
def pred_and_plot(model,image_path):
  img=mpimg.imread(image_path)
  img=tf.image.resize(img,size=[224,224])
  img=img[tf.newaxis,...]
  pred=model.predict(img)

  plt.figure(figsize=(9,7))
  plt.subplot(1,2,1)
  plt.imshow(img[0]/255)
  plt.title(f"Prediction:{class_names[np.argmax(pred)]}")
  plt.axis(False)
  plt.subplot(1,2,2)
  plt.bar(class_names,pred[0])
  plt.title('probability')
  plt.show()

pred_and_plot(model,'/content/myimg2.jpg')

In [ ]:
def pred_and_plot(model,image_path):
  img=mpimg.imread(image_path)
  img=tf.image.resize(img,size=[224,224])
  img=img[tf.newaxis,...]
  pred=model.predict(img)

  plt.figure(figsize=(8,5))
  plt.subplot(1,2,1)
  plt.imshow(img[0]/255)
  plt.title(f"Prediction:{class_names[np.argmax(pred)]}")
  plt.axis(False)
  plt.subplot(1,2,2)
  plt.bar(class_names,pred[0])
  plt.title('probability')
  plt.show()

pred_and_plot(model,'/content/img3.jpg')

In [ ]:
def pred_and_plot(model,image_path):
  img=mpimg.imread(image_path)
  img=tf.image.resize(img,size=[224,224])
  img=img[tf.newaxis,...]
  pred=model.predict(img)

  plt.figure(figsize=(8,5))
  plt.subplot(1,2,1)
  plt.imshow(img[0]/255)
  plt.title(f"Prediction:{class_names[np.argmax(pred)]}")
  plt.axis(False)
  plt.subplot(1,2,2)
  plt.bar(class_names,pred[0])
  plt.title('probability')
  plt.show()

pred_and_plot(model,'/content/myimg5.jpg')

In [ ]:
model.save('emotions_predictions.keras')

In [ ]:
import zipfile


model_file = 'emotions_predictions.keras'
zip_file = 'emotions_predictions.keras.zip'


with zipfile.ZipFile(zip_file, 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipf.write(model_file, arcname='emotions_predictions.keras')
